In [2]:
%config IPCompleter.greedy = True

In [3]:
import numpy as np

import torch
from torch import nn, optim
import torch.nn.functional as F

from torchvision import datasets, transforms, models

import matplotlib.pyplot as plt
%matplotlib inline

from collections import OrderedDict
import time

In [4]:
train_transform = transforms.Compose([
                                    transforms.Resize(255),
                                    transforms.RandomCrop(224),
                                    transforms.RandomRotation(30),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                    
                                    ])

train_image_folder_path = 'Cat_Dog_data/train'
train_set = datasets.ImageFolder(train_image_folder_path, transform = train_transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size = 64, shuffle = True)

In [5]:
test_transform = transforms.Compose([
                                    transforms.Resize(255),
                                    transforms.RandomCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                    ])

test_image_folder_path = 'Cat_Dog_data/test'
test_set = datasets.ImageFolder(test_image_folder_path, transform = test_transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size = 64, shuffle = True)

In [6]:
model = models.densenet121(pretrained = True)
print(model)

C:\Users\pt250154\AppData\Local\Continuum\anaconda3\lib\site-packages\torchvision\models\densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

In [171]:
for param in model.parameters():
    param.requires_grad = False

In [172]:
classifier = nn.Sequential(OrderedDict([
        ('fc1', nn.Linear(1024, 500)),
        ('relu1', nn.ReLU()),
        ('fc2', nn.Linear(500, 2)),
        ('output', nn.LogSoftmax(dim = 1))
]))
model.classifier = classifier

In [173]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters())

In [174]:
def validation(model, test_loader, criterion):
    
    test_loss = 0
    accuracy = 0
    test_loader_length = len(test_loader)

    for ii, (images, labels) in enumerate(test_loader):
        
        print('{:2d}/{:2d} iteration of test_loader '.format(ii, test_loader_length))
        
        output = model.forward(images)
        ps = torch.exp(output)
        
        test_loss += criterion(output, labels).item()
        
        equality = (labels.data == ps.max(dim = 1)[1])
        accuracy += equality.type(torch.FloatTensor).mean()
        
    return test_loss, accuracy

In [175]:
start = time.time()
steps = 0
epochs = 3
train_loader_length = len(train_loader)

model.train()
for e in range(epochs):
    for images, labels in train_loader:
        steps +=1

        outputs = model.forward(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()
        
        print('Epochs: {:2d}/{:2d} | {:4d}/{:4d} iteration of train_loader'.format(e, epochs, steps, train_loader_length))

model.eval()
with torch.no_grad():
    test_loss, accuracy = validation(model, test_loader, criterion)
    print(
        'test_loss: {:.3f}'.format(test_loss/len(test_loader)),
        'accuracy: {:.3f}'.format(accuracy/len(test_loader))
    )
model.train()

Epochs:  0/ 3 |    1/ 352 iteration of train_loader
Epochs:  0/ 3 |    2/ 352 iteration of train_loader
Epochs:  0/ 3 |    3/ 352 iteration of train_loader
Epochs:  0/ 3 |    4/ 352 iteration of train_loader
Epochs:  0/ 3 |    5/ 352 iteration of train_loader
Epochs:  0/ 3 |    6/ 352 iteration of train_loader
Epochs:  0/ 3 |    7/ 352 iteration of train_loader
Epochs:  0/ 3 |    8/ 352 iteration of train_loader
Epochs:  0/ 3 |    9/ 352 iteration of train_loader
Epochs:  0/ 3 |   10/ 352 iteration of train_loader
Epochs:  0/ 3 |   11/ 352 iteration of train_loader
Epochs:  0/ 3 |   12/ 352 iteration of train_loader
Epochs:  0/ 3 |   13/ 352 iteration of train_loader
Epochs:  0/ 3 |   14/ 352 iteration of train_loader
Epochs:  0/ 3 |   15/ 352 iteration of train_loader
Epochs:  0/ 3 |   16/ 352 iteration of train_loader
Epochs:  0/ 3 |   17/ 352 iteration of train_loader
Epochs:  0/ 3 |   18/ 352 iteration of train_loader
Epochs:  0/ 3 |   19/ 352 iteration of train_loader
Epochs:  0/ 

KeyboardInterrupt: 